TODO: 大模型对每节内容进行纠正、分块

In [16]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [2]:
prompt = '''给定文本文档，从文本中识别出文档组织结构(即各级标题及正文内容)，并更正文档中可能存在的识别错误，剔除文档中的例题。

-步骤- 
1.统一文档中各级标题的形式，将标题格式化为如下形式：(<title>{tuple_delimiter}<title_level>{tuple_delimiter}<title_name>)
-title_level：标题级别，按照文档组织结构识别，为0-3的整数，0表示文档的最高级别标题，3表示文档的最低级别标题
-title_name：标题，结合上下文从文档中提取

2.按照步骤1提取的标题划分文档，得到分块的正文段落，以如下形式格式化正文段落：(<title>{tuple_delimiter}<title_level>{tuple_delimiter}<paragraphs>)
-paragraphs：正文内容块包含的所有文本信息

3.对于每个正文块，识别出文本中包含的*习题与解析*内容，从块的内容中删去这一部分，得到修改后的<clear_paragraphs>。

4.对于修改后的<clear_paragraphs>，识别文档中可能存在的问题，并修正，确保没有缺字漏字现象。

5.完成后，输出格式化的文档，包含按原始顺序排列的格式化标题和正文内容。

-注意事项-
1.文档中可能存在的错误：文档中可能存在的错误包括缺字漏字、错别字、文档结构不规范等。
2.文档中可能存在的习题与解析：文档中可能存在 即学即练、走近真题、【名家精析】、【参考答案】等部分，这些部分是需要被去除的。
3.文档中可能存在以latex公式形式出现的标号或文本格式，如$\textcircled{{1}}$、$\\bigcirc$、需要更正为正常文本形式
4.尽量忠实原文，不要对原文进行过多的修改，保持原文的完整性
5.格外需要注意的是标题通常以第一节 一、 (一) 1.的形式出现，正文中出现的这样的文本应该被识别为标题
6.两个标题之间的文本应该被合并为一个段落，不应该被分割为两个段落

-示例-
######################
示例1：
输入文本文档:
第一节 教育制度概述 小库提醒 小库导学 小库课堂 一、制度的概念 【单选】 关于制度的内涵中外学者各抒己见，有多种说法。
综合中西方关于制度的认识，我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。
二、教育制度的定义与构成要素 【单选、填空】
(一)教育制度的定义 教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。
(1《教育大辞典》的界定是“一个国家各种教育机构的体系。包括学校制度和管理学校的教育行政机构体系”。
(2)《中国大百科全书》教育卷)对教育制度作出了两种阐释：一是“根据国家的性质制定的教育目的、方针和设施的总称”；二是“各种教育机构系统”。
(3)在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：$\textcircled{{1}}$ 根据国家的性质制定的教育目的、方针和设施的总称。 $\circledcirc$ 一个国家内各种教育机构的体系：一是泛指有组织的教育和教学的机构体系，包括学前教育机构、各级各类学校教育机构、成人教育机构、少年儿童校外教育机构以及各级教育行政组织机构；二是专指各级各类学校教育制度，简称学制。
(二)教育制度的构成要素 制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。关于制度的结构，许多学者给出了不同的答案。
有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。
1.规则要素 规则要素主要指教育制度是一个规范系统，用以规范学校成员的教育、教学行为，协调学校成员的关系以及教育与社会其他层面的关系。
2.物质要素 物质要素是指教育制度得以顺利运转所必须依赖的物质资料和设备。最常见的教育设备包括教室、桌椅、教具等。
随着科技的发展，新的教育设备在不断更新与采用，如计算机及多媒体设备在学校中普及就是一个典型例子。
教育制度作为基本的社会结构的一个组成部分，是一种有组织的社会力量。现代学校教育制度的成型，实际上是在社会发展变化过程中教育制度化的变化、发展、演进的结果。
即学即练 (单选)英国政府1870年颁布的《初等教育法》，一方面保持原有的专为资产阶级子女服务的学校系统，另一方面为劳动人民的子女设立国民小学、职业学校。这种学制属于
A.双轨型学制 B.单轨型学制C.中间型学制 D.分支型学制【名家精析】A双轨型学制是在 $1\,8\sim1\,9$ 世纪特定历史背景下，为了保障资产阶级子弟教育特权，同时满足劳动者子女教育文化需求的条件下而产生的学校系统。双轨型学制以第二次世界大战前的英、法、德等西欧国家为代表。
################
输出:
(<title>{tuple_delimiter}<0>{tuple_delimiter}<教育制度概述>)
(<title>{tuple_delimiter}<1>{tuple_delimiter}<制度的概念>)
(<text>{tuple_delimiter}<关于制度的内涵中外学者各抒己见，有多种说法。综合中西方关于制度的认识，我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。>)
(<title>{tuple_delimiter}<1>{tuple_delimiter}<教育制度的定义与构成要素>)
(<title>{tuple_delimiter}<2>{tuple_delimiter}<教育制度的定义>)
(<text>{tuple_delimiter}<教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。(1)《教育大辞典》的对教育制度的界定是“一个国家各种教育机构的体系。包括学校制度和管理学校的教育行政机构体系”。(2)《中国大百科全书》(教育卷)对教育制度作出了两种阐释：一是“根据国家的性质制定的教育目的、方针和设施的总称”；二是“各种教育机构系统”。(3)在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：①根据国家的性质制定的教育目的、方针和设施的总称。②一个国家内各种教育机构的体系：一是泛指有组织的教育和教学的机构体系，包括学前教育机构、各级各类学校教育机构、成人教育机构、少年儿童校外教育机构以及各级教育行政组织机构；二是专指各级各类学校教育制度，简称学制。>)
(<title>{tuple_delimiter}<2>{tuple_delimiter}<教育制度的构成要素>)
(<text>{tuple_delimiter}<制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。关于制度的结构，许多学者给出了不同的答案。有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。>)
(<title>{tuple_delimiter}<3>{tuple_delimiter}<教育制度的规则要素>)
(<text>{tuple_delimiter}<规则要素主要指教育制度是一个规范系统，用以规范学校成员的教育、教学行为，协调学校成员的关系以及教育与社会其他层面的关系。这一规范系统可以分为多种层次与范围，一般涉及成文的规范与不成文的规范。成文的规范可以体现为教育法律、教育法规、教育规章等；不成文的规范主要是指教育习俗。规范系统出现紊乱的时候，就可能导致教育功能失调。在社会变迁过程中，教育规范系统会发生变化，一方面原有的规范系统会朝着简化的方向发展；另一方面随着新的教育问题的出现，又会出现许多新的教育规范。>)
(<title>{tuple_delimiter}<3>{tuple_delimiter}<教育制度的物质要素>)
(<text>{tuple_delimiter}<物质要素是指教育制度得以顺利运转所必须依赖的物质资料和设备。最常见的教育设备包括教室、桌椅、教具等。随着科技的发展，新的教育设备在不断更新与采用，如计算机及多媒体设备在学校中普及就是一个典型例子。教育制度作为基本的社会结构的一个组成部分，是一种有组织的社会力量。现代学校教育制度的成型，实际上是在社会发展变化过程中教育制度化的变化、发展、演进的结果。>)'''

In [3]:
system_template="""
-目标- 
给定文本文档，从文本中识别出文档组织结构(即各级标题及正文内容)，并更正文档中可能存在的识别错误，剔除文档中的例题。

-步骤- 
1.统一文档中各级标题的形式，将标题格式化为如下形式：(title{tuple_delimiter}title_level{tuple_delimiter}title_name)
-title_level：标题级别，按照文档组织结构识别，为0-4的整数，0表示文档的最高级别标题，4表示文档的最低级别标题。
-title_name：标题，结合上下文从文档中提取
需要识别的标题形式为：第一节 xxx | 一、xxx | (一) xxx | 1.xxx | (1)xxx 五种，分别对应标题级别0-4

2.按照步骤1提取的标题划分文档，得到分块的正文段落，以如下形式格式化正文段落：(title{tuple_delimiter}title_level{tuple_delimiter}paragraphs)
-paragraphs：正文内容块包含的所有文本信息

3.对于每个正文块，识别出文本中包含的*习题与解析*内容，从块的内容中删去这一部分，得到修改后的<clear_paragraphs>。

4.对于修改后的<clear_paragraphs>，识别文档中可能存在的问题，并修正，确保没有缺字漏字现象。

5.完成后，输出格式化的文档，包含按原始顺序排列的格式化标题和正文内容。

-注意事项-
1.文档中可能存在的错误：文档中可能存在的错误包括缺字漏字、错别字、文档结构不规范等。
2.文档中可能存在的习题与解析：文档中可能存在 即学即练、走近真题、【名家精析】、【参考答案】等部分，这些部分是需要被去除的。
3.文档中可能存在以latex公式形式出现的标号或文本格式，如$\textcircled{{1}}$、$\\bigcirc$、需要更正为正常文本形式

-示例-
######################
示例1：
输入文本文档:
第一节 教育制度概述 一、制度的概念 【单选】 关于制度的内涵中外学者各抒己见，有多种说法。
我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。
二、教育制度的定义与构成要素 【单选、填空】
(一)教育制度的定义 教育制度是一个应用十分广泛的概念。
(1《教育大辞典》的界定是“一个国家各种教育机构的体系。包括学校制度和管理学校的教育行政机构体系”。
(2)在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：$\textcircled{{1}}$ 根据国家的性质制定的教育目的、方针和设施的总称。 $\circledcirc$ 一个国家内各种教育机构的体系。
(二)教育制度的构成要素 制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。
有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。
1.规则要素 规则要素主要指教育制度是一个规范系统。
2.物质要素 物质要素是指教育制度得以顺利运转所必须依赖的物质资料和设备。
随着科技的发展，新的教育设备在不断更新与采用，如计算机及多媒体设备在学校中普及就是一个典型例子。
3.组织要素 教育组织和管理系统是教育制度实现其功能的外在表现形式，用以检查和推动教育制度的执行。
即学即练 (单选)英国政府1870年颁布的《初等教育法》属于
A.双轨型学制 B.单轨型学制C.中间型学制 D.分支型学制【名家精析】A双轨型学制是为了保障资产阶级子弟教育特权，同时满足劳动者子女教育文化需求的条件下而产生的学校系统。
################
输出:
(title{tuple_delimiter}0{tuple_delimiter}教育制度概述)
(title{tuple_delimiter}1{tuple_delimiter}制度的概念【单选】)
(text{tuple_delimiter}关于制度的内涵中外学者各抒己见，有多种说法。我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。)
(title{tuple_delimiter}1{tuple_delimiter}教育制度的定义与构成要素【单选、填空】)
(title{tuple_delimiter}2{tuple_delimiter}教育制度的定义)
(text{tuple_delimiter}教育制度是一个应用十分广泛的概念。)
(title{tuple_delimiter}4{tuple_delimiter}《教育大辞典》对教育制度的定义)
(text{tuple_delimiter}《教育大辞典》对教育制度的界定是“一个国家各种教育机构的体系。包括学校制度和管理学校的教育行政机构体系”。)
(title{tuple_delimiter}4{tuple_delimiter}《教育百科辞典》对教育制度的定义)
(text{tuple_delimiter}在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：①根据国家的性质制定的教育目的、方针和设施的总称。②一个国家内各种教育机构的体系。)
(title{tuple_delimiter}2{tuple_delimiter}教育制度的构成要素)
(text{tuple_delimiter}制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。)
(title{tuple_delimiter}3{tuple_delimiter}教育制度的规则要素)
(text{tuple_delimiter}规则要素主要指教育制度是一个规范系统。)
(title{tuple_delimiter}3{tuple_delimiter}教育制度的物质要素)
(text{tuple_delimiter}物质要素是指教育制度得以顺利运转所必须依赖的物质资料和设备。随着科技的发展，新的教育设备在不断更新与采用，如计算机及多媒体设备在学校中普及就是一个典型例子。)
(title{tuple_delimiter}3{tuple_delimiter}教育制度的组织要素)
(text{tuple_delimiter}教育组织和管理系统是教育制度实现其功能的外在表现形式，用以检查和推动教育制度的执行。)
######################
示例2：
输入文本文档:
(四)启发性原则 启发性原则反映了学生的认识规律，学生的认识过程是在教师指导下进行的能动认识过程。
######################
输出:
(title{tuple_delimiter}2{tuple_delimiter}启发性原则)
(text{tuple_delimiter}启发性原则反映了学生的认识规律，学生的认识过程是在教师指导下进行的能动认识过程。)
######################
示例3：
输入文本文档:
3.教育制度的定义 教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。
######################
输出:
(title{tuple_delimiter}3{tuple_delimiter}教育制度的定义)
(text{tuple_delimiter}教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。)
-输入-

"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="""
-真实数据- 
######################
文本：{input_text} 
###################### 
输出：
"""

from langchain.chains import LLMChain
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_template])

chain = LLMChain(llm=llm, prompt=chat_prompt)
# with open(r"D:\code\agent_demo\kg\output\1_第一节 教育概述.txt", "r") as f:
#     input = f.read()
#     input = input.replace('{', '{{').replace('}', '}}')
#     answer = chain.invoke({
#         "tuple_delimiter": '#-#-#-#',
#         "input_text": input
#         })
#     print(answer['text'])
    
#     # 保存
#     with open(r"D:\code\agent_demo\kg\outv2\1_第一节 教育概述_1.txt", "w") as f:
#         f.write(answer['text'])

C:\Users\pixiu\AppData\Local\Temp\ipykernel_20048\2413214506.py:95: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=chat_prompt)


In [4]:
#* 改错的提示词
# 备选方案
# 划分

In [5]:
# 遍历文件夹的txt文件
import re
def get_files(path):
    import os
    files = os.listdir(path)
    return [os.path.join(path, file) for file in files if file.endswith('.txt')]

def split_text(text):
    # 正则匹配 "一、"这种编号
    pattern = r"(?:(?<=\n)|(?<=\s)|(?<=^))([一二三四五六七八九十])、"
    parts = re.split(pattern, text)
    # print(parts)
    
    result = []
    content = parts[0]
    for i in range(1, len(parts), 2):
        tmp = parts[i] + '、' + parts[i + 1]
        
        if len(tmp) > 3000:
            pattern = r'(?=(\([一二三四五六七八九十]\)))'
            part = re.split(pattern, tmp)
            for j in range(0, len(part), 2):
                st = part[j]
                if len(content) + len(st) > 3000:
                    result.append(content)
                    content = st
                else:
                    content += st
        else:
            if len(content) + len(tmp) > 3000:
                result.append(content)
                content = tmp
            else:
                content += tmp

    result.append(content)
    # print(result)
    
    return result

# i  = 0
files = get_files(r"D:\code\agent_demo\kg\output")
for file in files:
    with open(file, "r") as f:
        input = f.read()
        input = input.replace('{', '{{').replace('}', '}}')
        if len(input) > 3000:
        # 如果文本太长就切开控制文档长度 一、二、三
        # 如果还是太长就切成更小的文本 （一）（二）（三）
        # 效果不好 试试每一个度切成段落 （不要让chunk过多）
            inputs = split_text(input)
        else:
            inputs = [input]
        # print(inputs)
        
        # if i == 0 :
        #     i = 1
        fle = ""
        for input in inputs:
            answer = chain.invoke({
                "tuple_delimiter": '-#-#-',
                "input_text": input
                })
            print(answer['text'])
            fle += answer['text']
        
        # 保存
        with open(file.replace('output', 'output_4o_mini'), "w") as f:
            f.write(fle)

(title-#-#-0-#-#-教育制度概述)
(title-#-#-1-#-#-制度的概念【单选】)
(text-#-#-关于制度的内涵中外学者各抒己见，有多种说法。综合中西方关于制度的认识，我们认为制度指为了满足人们的生存与发展的需要、在基于权利和义务的基础上形成的社会关系，以及与此相关的社会活动的规范系统。制度实际上包含三层含义：1.指社会形态意义上的制度，这是对制度的宏观分析。如奴隶制度、封建制度、资本主义制度等，以整个社会作为实体，在区分社会发展类型、性质和阶段时使用。2.指社会中一些具体制度，这是对社会制度的中观分析。如一个社会内部的家庭、经济、政治、教育、宗教制度等，它以具体的社会组织机构、制度设施作为实体，常在分析不同的社会关系、研究不同社会生活领域的具体问题时使用。3.指社会规范、规则意义上的制度，是对制度的微观分析。如教育制度内部的考试制度、学习制度等，它以具体社会单位的行为规范、行为模式为实体，常在讨论个体行为时使用。)
(title-#-#-1-#-#-教育制度的定义与构成要素【单选、填空】)
(title-#-#-2-#-#-教育制度的定义)
(text-#-#-教育制度是一个应用十分广泛的概念，研究者给出的定义也丰富多彩。《教育大辞典》的界定是“一个国家各种教育机构的体系，包括学校制度和管理学校的教育行政机构体系”。《中国大百科全书》教育卷对教育制度作出了两种阐释：一是“根据国家的性质制定的教育目的、方针和设施的总称”；二是“各种教育机构系统”。在大陆和台湾学者联合编写的《教育百科辞典》中，教育制度有两种解释：根据国家的性质制定的教育目的、方针和设施的总称；一个国家内各种教育机构的体系：一是泛指有组织的教育和教学的机构体系，包括学前教育机构、各级各类学校教育机构、成人教育机构、少年儿童校外教育机构以及各级教育行政组织机构；二是专指各级各类学校教育制度，简称学制。)
(title-#-#-2-#-#-教育制度的构成要素)
(text-#-#-制度是由组成制度的各个要素相互影响、作用与结合而构成的有机整体。关于制度的结构，许多学者给出了不同的答案。有研究者认为制度包括制度目标、制度规则、制度组织和制度载体四个基本要素。而诺斯根据制度的不同表现形式，将制度分为正式约束的规则、非正式约束的规则和实施机制三个部分。我们认为，作为一种规范系统，教育

KeyboardInterrupt: 

In [25]:
import os
from langchain_neo4j import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://fed64bff.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "O8tsGxq0Zr-2rTV3CckvG5tflwyXeWHt6tHvws0ziow"

graph = Neo4jGraph()

In [26]:
def get_decs_from_text(title, text):
    # 从文本中提取描述
    decs_sys_template = """
-目标- 
给定一个标题和与标题相关的文本文档，以不超过150字描述文档内容，为标题提供一个简短的描述。

-注意事项-
确保描述简洁明了、不存在指代混乱以及冗余。

-示例-
######################
示例1：
标题：自然成熟理论
文本：自然成熟理论的代表人物是格赛尔。他认为支配儿童发展的因素是成熟和学习，他更着重于成熟对儿童发展的制约，认为成熟与主体的内环境有关，而学习则与客观外环境有关。儿童心理发展是儿童行为或心理形式在环境的影响下按一定顺序出现的过程，这个顺序与成熟关系较大，而与外环境关系较小，外环境只是给发展提供适应的时机而已。格赛尔指出无视成熟的学习和训练是徒劳的，是没有成效的，特别是人的早期发展，对内部成熟的依赖程度大大高于外部环境的学习和影响。他的“双生子爬梯试验”提供了儿童发展中成熟优势的实证，为大家所公认。此实验说明了成熟是某些行为、能力产生的必需条件，是身心发展的一种准备状态。在某些人体机能的生理结构未成熟之前，学习和训练的成效是很差的，甚至是不可能进行的，只有在某一行为出现的机体发育临近或达到成熟状态时，学习和训练才能奏效。
——————————————————————
输出：自然成熟理论由格赛尔提出，强调儿童发展主要受生理成熟驱动，外部环境仅提供适应机会。格赛尔认为儿童心理与行为的发展遵循固定顺序，受成熟影响较大，而学习作用有限。格赛尔的“双生子爬梯试验”证明，某些能力的形成依赖生理成熟，若机体未达成熟状态，学习与训练难以奏效。
######################
示例2：
标题：研究报告的结构
文本：研究报告的结构包括以下几个部分：题目、摘要、导言、文献综述、方法与设计、研究结果、结论与建议、参考文献。其中，导言部分包括：①研究的背景，包括与研究领域有关的历史观点，研究领域的主要议题、问题和优点并指出研究趋势。②研究问题，包括确认研究的基本议题，特别指出这些议题的各个方面以及现存知识的不足，由此发展出主要的研究问题。③研究意义，一般可以从理论与实践意义两个层面分而述之。研究报告从问题的确定开始，到结论和启示结束。一般，毕业或学位论文多采用分章的形式，而期刊文章则主要以标题来组织。
——————————————————————
输出：研究报告通常包括题目、摘要、导言、文献综述、方法与设计、研究结果、结论与建议及参考文献。导言部分涵盖研究背景、研究问题及研究意义，明确领域现状、研究焦点及价值。研究报告由问题提出至结论形成。毕业论文多采用分章形式，而期刊文章则以标题组织内容。
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    -真实数据- 
    ######################
    标题：{title}
    文本：{input_text} 
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])

    chain = LLMChain(llm=llm, prompt=chat_prompt)
    
    answer = chain.invoke({
        "title": title,
        "input_text": text
        })
    
    return answer['text']

# title = "教育的词源"
# text = '在西方，教育源于拉丁语educere一词，是引导和启发之意。在西方，很多教育家强调教育是一种顺其自然的活动，旨在把自然人所固有的或潜在的素质由内而外引发出来。我国古代，在相当长的历史时期里，“教”和“育”一直是分开使用的，最早把“教”和“育”二字合为一体的人是孟子。《孟子·尽心上》中写道：“君子有三乐，而王天下不与存焉。父母俱存，兄弟无故，一乐也；仰不愧于天，俯不作于人，二乐也；得天下英才而教育之，三乐也。”东汉时期，许慎的《说文解字》将“教”和“育”释义为：“教，上所施，下所效也”“育，养子使作善也”。这是对“教育”一词的最早解释。“上所施”是指在上者如父母、兄长或老师等人的传授、讲解、施教的活动；“下所效”是指在下者如子女、晚辈或学生学习、听讲、模仿、仿效的活动；“养子使作善”即教育子女，使他们向好的方向发展。“教育”一词最早由孟子提出，出自《孟子·尽心上》；最早由许慎进行解释，出自《说文解字》。'
# print(get_decs_from_text(title, text))

In [27]:
def get_decs_from_text(title, text):
    # 从文本中提取描述
    decs_sys_template = """
-目标- 
给定一个标题和与标题相关的文本文档，以不超过150字描述文档内容，为标题提供一个简短的描述。

-注意事项-
确保描述简洁明了、不存在指代混乱以及冗余。

-示例-
######################
示例1：
标题：自然成熟理论
文本：自然成熟理论的代表人物是格赛尔。他认为支配儿童发展的因素是成熟和学习，他更着重于成熟对儿童发展的制约，认为成熟与主体的内环境有关，而学习则与客观外环境有关。儿童心理发展是儿童行为或心理形式在环境的影响下按一定顺序出现的过程，这个顺序与成熟关系较大，而与外环境关系较小，外环境只是给发展提供适应的时机而已。格赛尔指出无视成熟的学习和训练是徒劳的，是没有成效的，特别是人的早期发展，对内部成熟的依赖程度大大高于外部环境的学习和影响。他的“双生子爬梯试验”提供了儿童发展中成熟优势的实证，为大家所公认。此实验说明了成熟是某些行为、能力产生的必需条件，是身心发展的一种准备状态。在某些人体机能的生理结构未成熟之前，学习和训练的成效是很差的，甚至是不可能进行的，只有在某一行为出现的机体发育临近或达到成熟状态时，学习和训练才能奏效。
——————————————————————
输出：自然成熟理论由格赛尔提出，强调儿童发展主要受生理成熟驱动，外部环境仅提供适应机会。格赛尔认为儿童心理与行为的发展遵循固定顺序，受成熟影响较大，而学习作用有限。格赛尔的“双生子爬梯试验”证明，某些能力的形成依赖生理成熟，若机体未达成熟状态，学习与训练难以奏效。
######################
示例2：
标题：研究报告的结构
文本：研究报告的结构包括以下几个部分：题目、摘要、导言、文献综述、方法与设计、研究结果、结论与建议、参考文献。其中，导言部分包括：①研究的背景，包括与研究领域有关的历史观点，研究领域的主要议题、问题和优点并指出研究趋势。②研究问题，包括确认研究的基本议题，特别指出这些议题的各个方面以及现存知识的不足，由此发展出主要的研究问题。③研究意义，一般可以从理论与实践意义两个层面分而述之。研究报告从问题的确定开始，到结论和启示结束。一般，毕业或学位论文多采用分章的形式，而期刊文章则主要以标题来组织。
——————————————————————
输出：研究报告通常包括题目、摘要、导言、文献综述、方法与设计、研究结果、结论与建议及参考文献。导言部分涵盖研究背景、研究问题及研究意义，明确领域现状、研究焦点及价值。研究报告由问题提出至结论形成。毕业论文多采用分章形式，而期刊文章则以标题组织内容。
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    -真实数据- 
    ######################
    标题：{title}
    文本：{input_text} 
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])

    chain = LLMChain(llm=llm, prompt=chat_prompt)
    
    answer = chain.invoke({
        "title": title,
        "input_text": text
        })
    
    return answer['text']

# title = "教育的词源"
# text = '在西方，教育源于拉丁语educere一词，是引导和启发之意。在西方，很多教育家强调教育是一种顺其自然的活动，旨在把自然人所固有的或潜在的素质由内而外引发出来。我国古代，在相当长的历史时期里，“教”和“育”一直是分开使用的，最早把“教”和“育”二字合为一体的人是孟子。《孟子·尽心上》中写道：“君子有三乐，而王天下不与存焉。父母俱存，兄弟无故，一乐也；仰不愧于天，俯不作于人，二乐也；得天下英才而教育之，三乐也。”东汉时期，许慎的《说文解字》将“教”和“育”释义为：“教，上所施，下所效也”“育，养子使作善也”。这是对“教育”一词的最早解释。“上所施”是指在上者如父母、兄长或老师等人的传授、讲解、施教的活动；“下所效”是指在下者如子女、晚辈或学生学习、听讲、模仿、仿效的活动；“养子使作善”即教育子女，使他们向好的方向发展。“教育”一词最早由孟子提出，出自《孟子·尽心上》；最早由许慎进行解释，出自《说文解字》。'
# print(get_decs_from_text(title, text))

In [23]:
import gc
gc.collect() 

9232

In [ ]:
import re
cnt = 1
def parse_text(input_text):
    title_pattern = re.compile(r'\(title-#-#-(\d+)-#-#-(.*?)\)')
    text_pattern = re.compile(r'\(text-#-#-(.*?)\)')
    
    titles = list(title_pattern.finditer(input_text))
    texts = list(text_pattern.finditer(input_text))
    
    results = []
    title_list = []
    text_dict = {}
    
    for title in titles:
        level = int(title.group(1))
        name = title.group(2)
        title_list.append((name, level, title.start()))
    
    for text in texts:
        text_content = text.group(1)
        text_dict[text.start()] = text_content
    
    id_map = {}
    prev_stack = []
    
    for i, (name, level, start) in enumerate(title_list):
        next_start = title_list[i + 1][2] if i + 1 < len(title_list) else float('inf')
        collected_text = ""
        
        for text_start in sorted(text_dict.keys()):
            if start < text_start < next_start:
                collected_text += text_dict[text_start] + "\n"
        
        # 分配 ID
        global cnt
        title_id = cnt
        cnt += 1
        id_map[start] = title_id
        
        # 查找 pre 值
        while prev_stack and prev_stack[-1][1] >= level:
            prev_stack.pop()
        pre_id = prev_stack[-1][0] if prev_stack else None
        prev_stack.append((title_id, level))
        
        results.append((title_id, name, level, pre_id, collected_text.strip()))
    
    return results

import hashlib
from langchain_core.documents import Document
from typing import List

def init_main_concept(graph, chapter: str, titles: List)-> list:
    batch_data = []
    relationshipst2t = []
    relationshipst2c = []
    for i, item in enumerate(titles):
        id, name, level, pre_id, content = item
        page_content_sha1 = hashlib.sha1(content.encode())
        
        # if len(content) > 100:
        #     desc = get_decs_from_text(name, content)
        # else:
        #     desc = content
        
        title_data = {
            "id": id,
            "name": name,
            "level": level,
            "chapter": chapter,
            "page_content_sha1": page_content_sha1.hexdigest(),
            "page_content": content,
            # "description": desc
        }
        
        batch_data.append(title_data)
        
        if level == 0:
            relationshipst2c.append({"id": id, "chapter": chapter})
        else:
            relationshipst2t.append({"id": id, "pid": pre_id})

    # print(batch_data)
    print(relationshipst2t)
    print(relationshipst2c)
    
    query_to_create_main_concept = """
        UNWIND $batch_data AS data
        MERGE (c:`主要概念` {id: data.id})
        SET c += data
    """
    graph.query(query_to_create_main_concept, params={"batch_data": batch_data})
    
    query_to_create_PART_OF_relation = """
        UNWIND $relationshipst2t AS relationship
        MATCH (c:`主要概念` {id: relationship.id})
        MATCH (pc:`主要概念` {id: relationship.pid})
        MERGE (c)-[:PART_OF]->(pc)
        """
    graph.query(query_to_create_PART_OF_relation, params={"relationshipst2t": relationshipst2t})
    
    query_to_create_PART_OF_relation2 = """
        UNWIND $relationshipst2c AS relationship 
        MATCH (c:`主要概念` {id: relationship.id})
        MATCH (pc:`节` {title: relationship.chapter})
        MERGE (c)-[:PART_OF]->(pc)
        """
    graph.query(query_to_create_PART_OF_relation2, params={"relationshipst2c": relationshipst2c})
    
    return 0


files = get_files(r"D:\code\agent_demo\kg\outv2")
for file in files:
    with open(file, "r") as f:
        input = f.read()
        
        pattern = r'(?<=\d_)(.+)(?=\.\w+$)'  # 匹配 "1_XXX.txt" 中的 XXX
        match = re.search(pattern, file)

        if match:
            chapter = match.group(1)
        
        # print(chapter)
        
        results = parse_text(input)
        init_main_concept(graph, chapter, results)
        # for id, name, level, pre_id, text in results:
        #     print(f"{id} {name} {level} {pre_id} {text}")
        
        # 调整结构
        query_update_part_of = """
        MATCH (a:`主要概念` {name: $concept_name})
        MATCH (section:`节` {title: $section_title})
        MATCH (c)-[r:PART_OF]->(a)
        DELETE r
        MERGE (c)-[:PART_OF]->(section)
        SET section.page_content = a.page_content
        DETACH DELETE a
        """

        graph.query(query_update_part_of, params={"concept_name": chapter[4:], "section_title": chapter})
        print(chapter[4:])

        

[{'id': 2, 'pid': 1}, {'id': 3, 'pid': 1}, {'id': 4, 'pid': 3}, {'id': 5, 'pid': 4}, {'id': 6, 'pid': 4}, {'id': 7, 'pid': 4}, {'id': 8, 'pid': 3}, {'id': 9, 'pid': 8}, {'id': 10, 'pid': 8}, {'id': 11, 'pid': 8}, {'id': 12, 'pid': 1}, {'id': 13, 'pid': 12}, {'id': 14, 'pid': 12}, {'id': 15, 'pid': 12}, {'id': 16, 'pid': 12}, {'id': 17, 'pid': 16}, {'id': 18, 'pid': 16}, {'id': 19, 'pid': 16}]
[{'id': 1, 'chapter': '第一节 教育制度概述'}]
教育制度概述
[{'id': 21, 'pid': 20}, {'id': 22, 'pid': 21}, {'id': 23, 'pid': 22}, {'id': 24, 'pid': 22}, {'id': 25, 'pid': 22}, {'id': 26, 'pid': 22}, {'id': 27, 'pid': 21}, {'id': 28, 'pid': 27}, {'id': 29, 'pid': 27}, {'id': 30, 'pid': 29}, {'id': 31, 'pid': 29}, {'id': 32, 'pid': 29}, {'id': 33, 'pid': 29}, {'id': 34, 'pid': 20}, {'id': 35, 'pid': 34}, {'id': 36, 'pid': 34}, {'id': 37, 'pid': 36}, {'id': 38, 'pid': 37}, {'id': 39, 'pid': 37}, {'id': 40, 'pid': 37}, {'id': 41, 'pid': 37}, {'id': 42, 'pid': 36}, {'id': 43, 'pid': 42}, {'id': 44, 'pid': 42}, {'id': 

In [45]:
query_update_part_of = """
MATCH (a)-[r:CONTAINS]->(b)
WITH a, b, r
DELETE r
MERGE (a)-[:包括]->(b)
"""

graph.query(query_update_part_of)

[]

In [59]:
from neo4j import GraphDatabase

# 连接到 Neo4j
URI = "neo4j+s://fed64bff.databases.neo4j.io"  # 修改为你的 Neo4j 地址
AUTH = ("neo4j", "O8tsGxq0Zr-2rTV3CckvG5tflwyXeWHt6tHvws0ziow")  # 修改为你的用户名和密码

def add_unique_ids():
    query_chapter = """
    // 给 `章` 结点编号（从 2000 开始）
    MATCH (n:章)
    WITH n ORDER BY n.name
    WITH collect(n) AS nodes
    UNWIND range(0, size(nodes) - 1) AS i
    WITH nodes[i] AS node, 2000 + i AS new_id
    SET node.id = new_id;
    """

    query_section = """
    // 给 `节` 结点编号（从 3000 开始）
    MATCH (n:节)
    WITH n ORDER BY n.name
    WITH collect(n) AS nodes
    UNWIND range(0, size(nodes) - 1) AS i
    WITH nodes[i] AS node, 3000 + i AS new_id
    SET node.id = new_id;
    """

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            # 执行给 `章` 结点编号的查询
            session.run(query_chapter)
            # 执行给 `节` 结点编号的查询
            session.run(query_section)
            print("成功为 `章` 和 `节` 结点添加唯一编号！")

# 执行编号赋值
add_unique_ids()


成功为 `章` 和 `节` 结点添加唯一编号！


In [ ]:
from neo4j import GraphDatabase

# 连接到 Neo4j
URI = "neo4j+s://fed64bff.databases.neo4j.io"  # 修改为你的 Neo4j 地址
AUTH = ("neo4j", "O8tsGxq0Zr-2rTV3CckvG5tflwyXeWHt6tHvws0ziow")  # 修改为你的用户名和密码

# 属于一个节的所有主要概念
def get_chapter_full_subgraph():
    query = """
    MATCH (chapter:`节`)
    CALL {
        WITH chapter
        MATCH path = (chapter)-[*]->(sub)
        RETURN collect({node: sub, relationships: relationships(path)}) AS subgraph
    }
    RETURN chapter.name AS chapter_title, subgraph
    """

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run(query)
            chapters = []
            for record in result:
                chapter_info = {
                    "chapter_title": record["chapter_title"],
                    "subgraph": []
                }
                for entry in record["subgraph"]:
                    node = entry["node"]
                    relationships = entry["relationships"]
                    chapter_info["subgraph"].append({
                        "node": dict(node),  # 获取所有节点属性
                        "relationships": [{"type": rel.type, "start": rel.start_node["id"], "end": rel.end_node["id"]} for rel in relationships]
                    })
                chapters.append(chapter_info)
            return chapters

# 获取所有章的完整子图
full_subgraphs = get_chapter_full_subgraph()

# 打印结果
import json
print(json.dumps(full_subgraphs, indent=4, ensure_ascii=False))

In [61]:
# 保存 json
with open(r"D:\code\agent_demo\kg\output\full_subgraphs.json", "w", encoding="utf-8") as f:
    json.dump(full_subgraphs, f, indent=4, ensure_ascii=False)